# Библиотека имитационного моделирования SimPy

Моделирование событийно-ориентированных процессов - в частности процесса поиска, диагностики и устранения неисправностей -  прдеставляет собой сложную и ресурсоемкую задачу. Возможные решения как правило подразумевают один из трех основных подходов или их комбинацию:

1. Активностно-ориентированная парадигма.
2. Событийно-ориентированная парадигма.
3. Процесс-ориентированная парадигма.

При этом моделирование разделяется на условно "повторяемые эксперименты" и анализ временных областей или "эксперименты протяженные во времени".

Одним из примером модели, может быть анализ обслуживания очередей заявок на ремонт, при котором работы поступают в случайное время, а время обслуживания заявок тоже является случайной функцией. В таких условиях время между началом работы и ее выполнением будет протяженной случайной функцией, коррелирующей с экспоненциальным или другим распределением.

## 1. Активностно-ориентированная парадигма.

При активностно-ориентированной парадигме модельное время разделяется на небольшие интервалы - значительно меньшие, чем порядок происходящих событий. Для событий имеющих разброс примерно в 20 секунд - рекомендуемый временной шаг может составлять 0.001 секунды. В каждый момент времени моделирующая программа отслеживает состояние всех процессов в системе, проверять возможное завершение каждого из них и перераспределять занятость различных систем.

Для примера выше в каждый момент проверяется пришла ли новая заявка на работу или нет? Если событие произошло - происходит проверка занят ли сервер работой - если занят - работа ставится в очередь, если нет - работа загружается сразу для выполнения на сервере. На следующем шаге вся последовательность повторяется.


## 2. Событийно-ориентированная парадигма.

Недостаток активностно-ориентированной парадигмы в очень медленном исполнении. Большая часть времени уходит на бесполезные проверки (ничего из работ не пришло, ничего не закончилось), что приводит к неоправданным затратам вычислительных ресурсов. Это серьезно осложняет исследование, т.к. часто симуляция должна захватывать протяженные промежутки времени.

Существует способ резко увеличить производительность. Вместо отработки равных по длительности промежутков времени можно использовать "таймер экспериментальной модели", который позволяет сразу переходить ко времени следующего события. В рассмотренной выше модели заявок может быть два основных события - появление новой заявки, либо окончание обработки старой. Поэтому вместо подробной пошаговой проверки событий можно использовать схему, когда вычисляется время следующего события (появление заявки, окончание работ по заявке) и системное время модели сразу переставляется на это время - после чего происходит цикл событий соответствующих произошедшему событию.

Событийно-ориентированная парадигма формализует эту идею. Набор событий хранится в виде событийного стека. Каждое новое событие что-то изменяет в стеке и генерирует новое событие. Программный код при этом каждый раз проверяет стек событий и переставляет системное время на то, которое должно случиться со следующем по очереди событием. При этом на каждой итерации не только обрабатывается какое-то событие, но и вычисляется время следующего события.

Иногда это требует подходяющей структуры организации данных, однако во многих событийно-ориентированных приложениях такая струткура может быть реализована как простой линейно-связанный список. Этого достаточно до тех пор, пока в очереди не так много событий (в рассматриваемом примере максимальный размер - 2).

## 3. Процесс-ориентированная парадигма.

Процесс-ориентированная парадигма ориентирована на многопоточные вычисления. Организация работы с процессами очень напоминает таковую в семействе операционных систем стандарта POSIX. Более того, некоторые модели используют механизм работы, основанный на unix-подобном взаимодействии процессов. Однако, такие системы достаточно тяжело создавать и поддерживать. Одними из первых языков программирования, которые позволяли работать в такой парадигме были  Simula и библиотека CSIM языка Си - поскольку позволяли манипуляции со стеком. При таком подходе данные могут храниться в виде стека, которым управляет менеджер событий. Преимуществом таких систем является более модульный код.

# Практические примеры

## Тикающие часы


Вы создаете события и планируете их на заданное время моделирования. События сортируются по приоритету, времени моделирования и возрастающему идентификатору события. У события также есть список обратных вызовов, которые выполняются, когда событие запускается и обрабатывается циклом событий. События также могут иметь возвращаемое значение. 

https://simpy.readthedocs.io/en/latest/topical_guides/simpy_basics.html


* процесс - генератор / функция
* среда
* событие



    

Создадим два таймера. один быстрый - каждые 0.5 единиц времени, другой медленный - каждую одну единицу времени.

Это частотные таймеры - казалось бы один процесс должен отставать от другого - но тут они просто срабатывают с разной частотой (f и 2f).

Поэтому быстрый процесс сработает дважды, а медленный - один раз.

In [8]:
>>> import simpy
>>> # что на входе? имя, тик?
>>> def clock(env, name, tick): # env получаем из simpy.Environment
...     while True:
...         print(name, env.now) # процесс печатает свое имя и который сейчас час
...         yield env.timeout(tick) # тут процесс возвращает ... функцию которая делает паузу на тик раз
...
>>> env = simpy.Environment() # создаем среду

In [9]:
>>> env.process(clock(env, 'fast', 0.5)) # быстрый процесс
>>> env.process(clock(env, 'slow', 1)) # медленный процесс
>>> env.run(until=2) # запускает процессы, пока внутреннее время не достигнет 2


fast 0
slow 0
fast 0.5
slow 1
fast 1.0
fast 1.5


In [10]:
>>> env.run(until=10) # запускает процессы, пока внутреннее время не достигнет 2


slow 2
fast 2.0
fast 2.5
slow 3
fast 3.0
fast 3.5
slow 4
fast 4.0
fast 4.5
slow 5
fast 5.0
fast 5.5
slow 6
fast 6.0
fast 6.5
slow 7
fast 7.0
fast 7.5
slow 8
fast 8.0
fast 8.5
slow 9
fast 9.0
fast 9.5


## Моделирование парковки


Пример функции автомобиля, который либо двигается либо запаркован.

env.now - текущее время
env.timeout(t) поспать t условных единиц времени

In [11]:
>>> def car(env):
...     while True:
...         print('Start parking at %d' % env.now)
...         parking_duration = 5
...         yield env.timeout(parking_duration)
...
...         print('Start driving at %d' % env.now)
...         trip_duration = 2
...         yield env.timeout(trip_duration)


In [12]:
>>> import simpy
>>> env = simpy.Environment()


In [13]:
>>> env.process(car(env))


<Process(car) object at 0x7f749c1f3220>

env.run(until=porog) - управляет срабатыванием асинхронных процессов, пока время не достигает значения porog.



In [14]:
>>> env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14
